# 1. Imports


In [6]:
import sys
sys.path.append('../../') 

import geopandas as gpd
from src.organized_datasets_creation.utils import resolve_nominatim_city_name
from src.organized_datasets_creation.utils import convert_nominatim_name_to_filename
from src.graph_layering.graph_layer_creator import GraphLayerController
import pandas as pd
from typing import cast
import os
from src.graph_layering.graph_layer_creator import SourceType
import warnings
from src.graph_layering.create_hetero_data import create_hetero_data

from tqdm import tqdm

import wandb.util
import wandb
import os

import numpy as np
from src.graph.create_osmnx_graph import OSMnxGraph
import json
from shapely.geometry import Point
from joblib import dump


from datetime import datetime
from sklearn.metrics import f1_score, roc_auc_score
from wandb.util import generate_id
from sklearn.linear_model import LogisticRegression
from src.training.train import train
from sklearn.preprocessing import StandardScaler

# 2. Setting up env variables and configs


In [7]:
WANDB_API_KEY= "56fbb1bf3740faae9e7ef1917dec1f3f5426909b"

In [8]:
# WANDB_API_KEY = os.environ.get("WANDB_API_KEY", None)
# assert (
#     WANDB_API_KEY is not None
# ), "WANDB_API_KEY is not set, did you forget it in the config file?"

In [9]:
pd.read_csv(f"../../data/downstream_tasks/zabka_shops/zabka_shops.csv")

,id,slug,openTime,closeTime,city,address,postcode,voivodeship,county,community,region,salesRegion,openTimeSeconds,closeTimeSeconds,lat,lng,services
0,ID06093,"ID06093,gdansk-jabloniowa-29a",06:00,23:00,Gdańsk,Jabłoniowa 29A,80-175,Pomorskie,Gdańsk,GDAŃSK (GMINA MIEJSKA),DS3,PS3.6.3,21600.0,82800.0,54.330567,18.557187,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
1,ID03871,"ID03871,gorzow-wielkopolski-obroncow-pokoju-38...",06:00,23:00,Gorzów Wielkopolski,Obrońców Pokoju 38 nr 38 I,66-400,Lubuskie,Gorzów Wielkopolski,GORZÓW WIELKOPOLSKI (GMINA MIE,DS2,PS2.5.3,21600.0,82800.0,52.764806,15.264941,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
2,ID06169,"ID06169,ruda-slaska-ul-niedurnego-45-lok-1",06:00,23:00,Ruda Śląska,ul. Niedurnego 45 lok. 1,41-709,Śląskie,Ruda Śląska,RUDA ŚLĄSKA (GMINA MIEJSKA),DS8,PS8.2.2,21600.0,82800.0,50.284140,18.876000,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
3,ID06264,"ID06264,warszawa-ul-zeganska-18",06:00,23:00,Warszawa,ul. Żegańska 18,04-713,Mazowieckie,Warszawa,"WARSZAWA (GMINA MIEJSKA, MIAST",DS5,PS5.4.1,21600.0,82800.0,52.205620,21.172909,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
4,ID05100,"ID05100,wejherowo-dworcowa-2",06:00,23:00,Wejherowo,Dworcowa 2,84-200,Pomorskie,wejherowski,WEJHEROWO (GMINA MIEJSKA),DS3,PS3.2.3,21600.0,82800.0,54.603632,18.228519,"BIH,DEN,GSM,KPO,LOT,ODP,PAC,RAC,REJ,TER,ZBC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9754,ID0A221,"ID0A221,kedzierzyn-kozle-ul-gliwicka-1a",06:00,23:00,Kędzierzyn-Koźle,UL. GLIWICKA 1A,47-224,Opolskie,kędzierzyńsko-kozielski,KĘDZIERZYN-KOŹLE (GMINA MIEJSK,DS7,PS7.5.1,21600.0,82800.0,50.336754,18.183524,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"
9755,ID0B739,"ID0B739,lewin-brzeski-ul-kosciuszki-37d",06:00,23:00,Lewin Brzeski,UL. KOŚCIUSZKI 37D,49-340,Opolskie,brzeski (woj. opolskie),LEWIN BRZESKI (GMINA MIEJSKO-W,DS7,PS7.5.5,21600.0,82800.0,50.751824,17.615581,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"
9756,ID0C796,"ID0C796,uniejow-ul-targowa-18",00:00,23:59,Uniejów,UL. Targowa 18,99-210,Łódzkie,poddębicki,UNIEJÓW (GMINA MIEJSKO-WIEJSKA,DS6,PS6.6.5,0.0,86340.0,51.976362,18.791962,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"
9757,ID0D004,"ID0D004,warszawa-ul-zamkowa-8-lok-u1",06:00,23:00,Warszawa,UL. Zamkowa 8 lok. U1 .,03-890,Mazowieckie,Warszawa,"WARSZAWA (GMINA MIEJSKA, MIAST",DS5,PS5.4.1,21600.0,82800.0,52.274640,21.081131,"DEN,GSM,KPO,ODP,PAC,RAC,REJ,TER,ZBC"


In [10]:
WANDB_API_KEY= "56fbb1bf3740faae9e7ef1917dec1f3f5426909b"

# general settings
ORGANIZED_HEXES_LOCATION = "../../data/organized-hexes"
ORGANIZED_GRAPHS_LOCATION = "../../data/organized_graphs"
OSMNX_ALL_ATTRIBUTES_LOCATION = (
    "../../data/osmnx_attributes.json"
)

# downstream task settings
ALL_ZABKAS_LOCATION = "../../data/downstream_tasks/zabka_shops/zabka_shops.csv"
ZABKA_DIR = "../../data/downstream_tasks/zabka_shops/"

TRAIN_SAVE_DIR = "../../gradient_logs/"

SWEEP_RUNS_COUNT = 50
EPOCHS = 300

ATTRIBUTES_CONFIGURATIONS = [
    # {
    #     "USE_ORTOPHOTO": False,
    #     "USE_HEXES_ATTRS": True,
    #     "USE_OSMNX_ATTRS": False,
    # },
    {
        "USE_ORTOPHOTO": False,
        "USE_HEXES_ATTRS": True,
        "USE_OSMNX_ATTRS": True,
    },
    {
        "USE_ORTOPHOTO": False,
        "USE_HEXES_ATTRS": False,
        "USE_OSMNX_ATTRS": True,
    },
]

WANDB_SWEEP_PARAMS_GRAPH_DATA = {
    "method": "bayes",
    "metric": {"name": "mean_f1", "goal": "maximize"},
    "parameters": {
        "hidden_channels": {"values": [10, 20, 30, 40, 50]},
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2,
        },
        "num_conv_layers": {"values": [1, 2, 3, 4, 5]},
        "lin_layer_size": {"values": [8, 16, 32, 64, 128]},
        "num_lin_layers": {"values": [0, 1, 2, 3, 4]},
        "weight_decay": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2,
        },
    },
}

WANDB_SWEEP_PARAMS_TABULAR_DATA = {
    "method": "bayes",
    "metric": {"name": "mean_f1", "goal": "maximize"},
    "parameters": {
        "solver_penalty": {
            "values": [
                "lbfgs;l2",
                "liblinear;l1",
                "liblinear;l2",
                "newton-cg;l2",
                "newton-cholesky;l2",
                "sag;l2",
                "saga;elasticnet",
                "saga;l1",
                "saga;l2",
            ]
        },
        "C": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1,
        },
    },
}

# 3. Loading zabka

The process includes removing unused columns and creating GeoSeries from raw X Y points


In [11]:
all_zabkas = pd.read_csv(ALL_ZABKAS_LOCATION)


def create_point(x):
    return Point(float(x[0]), float(x[1]))


geometry = all_zabkas[["lng", "lat"]].apply(create_point, axis=1)

gdf_zabka = gpd.GeoDataFrame(all_zabkas, geometry=geometry, crs="EPSG:4326")
gdf_zabka.drop(columns=["lng", "lat"], inplace=True)
gdf_zabka['zabka_count'] = 1

# 4. Displaying available cities


In [12]:
cities = ["Kraków", "Poznań", "Szczecin", "Wrocław", "Warszawa"]

# 5. Creating GeoDataFrames

The process of creation has following steps:

1. loading OSMNX nodes and edges
2. assigning accidents to OSMNX nodes
3. taking latest H9 resolution hexes
4. combining OSMNX nodes, OSMNX edges, hexes in a single dict and packing it inside gdfs_dict


In [13]:
def add_accidents_to_osmnx_nodes(
    zabkas: gpd.GeoDataFrame,
    nodes: gpd.GeoDataFrame,
    edges: gpd.GeoDataFrame,
    city_name: str,
):
    with open(OSMNX_ALL_ATTRIBUTES_LOCATION) as f:
        all_attributes = json.load(f)

    osmnx_graph = OSMnxGraph(
        zabkas.loc[
            zabkas["county"] == resolve_nominatim_city_name(city_name), :
        ],
        nodes,
        edges,
        all_attributes,
        y_column_name="zabka_count"
    )
    osmnx_graph._aggregate(element_type="node", aggregation_method="count")
    return osmnx_graph.gdf_nodes


def create_gdfs(city_name: str, accidents_gdf: gpd.GeoDataFrame = gdf_zabka):
    city_folder_name = convert_nominatim_name_to_filename(
        resolve_nominatim_city_name(city_name)
    )
    osmnx_nodes = gpd.read_parquet(
        os.path.join(ORGANIZED_GRAPHS_LOCATION, city_folder_name, "nodes.parquet")
    )
    osmnx_nodes = osmnx_nodes.reset_index()
    osmnx_nodes.index.names = ["node_id"]
    osmnx_nodes["x"] = osmnx_nodes["geometry"].x
    osmnx_nodes["y"] = osmnx_nodes["geometry"].y

    osmnx_edges = gpd.read_parquet(
        os.path.join(ORGANIZED_GRAPHS_LOCATION, city_folder_name, "edges.parquet")
    )
    osmnx_edges = osmnx_edges.reset_index().rename(columns={"index": "edge_id"})
    osmnx_edges.index.names = ["edge_id"]

    assert osmnx_nodes.crs == osmnx_edges.crs
    assert osmnx_nodes.crs == accidents_gdf.crs

    osmnx_nodes = add_accidents_to_osmnx_nodes(
        zabkas=accidents_gdf,
        nodes=osmnx_nodes,
        city_name=city_name,
        edges=osmnx_edges,
    )

    hexes_years_folder = os.path.join(ORGANIZED_HEXES_LOCATION, city_folder_name)

    subfolders = [
        int(f)
        for f in os.listdir(hexes_years_folder)
        if os.path.isdir(os.path.join(hexes_years_folder, f))
    ]
    highest_year = subfolders[np.argmax(subfolders)]

    hexes: gpd.GeoDataFrame = gpd.read_parquet(
        os.path.join(
            ORGANIZED_HEXES_LOCATION,
            f"{convert_nominatim_name_to_filename(resolve_nominatim_city_name(city_name))}/{highest_year}/h9/count-embedder/dataset.parquet",
        )
    )

    hexes = hexes.rename(columns={"region_id": "h3_id"}).rename_axis(
        "region_id", axis=0
    )

    return dict(osmnx_nodes=osmnx_nodes, osmnx_edges=osmnx_edges, hexes=hexes)


print("Creating gdfs...")
gdfs_dict = {city_name: create_gdfs(city_name+ ", Poland") for city_name in tqdm(cities)}

Creating gdfs...


100%|██████████| 5/5 [00:01<00:00,  2.99it/s]


# 6. Creating GraphLayerController for each of the cities

The creation is based on previously made GeoDataFrames. The controller is used to transfer accidents Y values from OSMNX nodes to hexes. It is also used to create complete graph data in case of graph datasets.


In [14]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for gdf_for_city in gdfs_dict.values():
        gdf_for_city["controller"] = GraphLayerController(
            gdf_for_city["hexes"],
            gdf_for_city["osmnx_nodes"],
            gdf_for_city["osmnx_edges"],
        )

# 7. Patching hexes

The y value (1 = accident occured, 0 = no accident) is assigned to each of the hexes according to its underlying OSMNX nodes


In [15]:
def patch_hexes_with_y(
    osmnx_nodes: gpd.GeoDataFrame,
    hexes: gpd.GeoDataFrame,
    controller: GraphLayerController,
):
    virtual_edges = controller.get_virtual_edges_to_hexes(SourceType.OSMNX_NODES)
    hexes_with_y = cast(
        gpd.GeoDataFrame,
        hexes.merge(
            virtual_edges.merge(osmnx_nodes, left_on="source_id", right_index=True)[
                ["region_id", "zabka_count"]
            ]
            .groupby("region_id")
            .sum(),
            left_index=True,
            right_index=True,
            how="left",
        ).fillna(0),
    )
    hexes_with_y["zabka"] = (hexes_with_y["zabka_count"] > 0).astype(int)
    hexes_with_y.drop(columns="zabka_count", inplace=True)
    controller.hexes_gdf = hexes_with_y
    controller._hexes_centroids_gdf = controller._create_hexes_centroids_gdf()

In [16]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for gdfs in gdfs_dict.values():
        patch_hexes_with_y(gdfs["osmnx_nodes"], gdfs["hexes"], gdfs["controller"])

# 8. Creating graph data

Graph data is used when we include OSMNX attributes and in turn maintain the graph structure of the data

The data is created only once for now just to create (train, val, test) folds labels for crossvalidation on graph-based versions of the task


In [17]:
def create_graph_data(
    osmnx_nodes,
    osmnx_edges,
    hexes,
    controller: GraphLayerController,
    use_hexes_attr: bool,
    use_ortophoto: bool,
):

    edges_attr_columns = osmnx_edges.columns[
        ~osmnx_edges.columns.isin(["u", "v", "key", "geometry"])
    ]
    nodes_attr_columns = osmnx_nodes.columns[
        ~osmnx_nodes.columns.isin(["geometry", "x", "y", "osmid", "zabka_count"])
    ]
    hexes_attr_columns = (
        hexes.columns[~hexes.columns.isin(["geometry", "h3_id"])]
        if use_hexes_attr
        else []
    )

    data = create_hetero_data(
        controller,
        hexes_attrs_columns_names=hexes_attr_columns,
        osmnx_edge_attrs_columns_names=edges_attr_columns,
        osmnx_node_attrs_columns_names=nodes_attr_columns,
        virtual_edge_attrs_columns_names=[],
        hexes_y_columns_names=["zabka"],
    )
    return data


graph_data_dict = {
    city_name: create_graph_data(**gdfs, use_ortophoto=True, use_hexes_attr=True)
    for city_name, gdfs in gdfs_dict.items()
}

# 9. Creating tabular data

Tabular data is used when we omit OSMNX attributes and in turn lose the graph structure of the data

No folds creation on tabular-based versions of the task - using simple leave-one-out


In [18]:
def create_tabular_data(
    hexes: pd.DataFrame,
    controller: GraphLayerController,
    use_hexes_attr: bool,
    use_ortophoto: bool,
):
    assert use_ortophoto or use_hexes_attr, "Provide at least one data source"

    hexes_attr_columns = (
        hexes.columns[~hexes.columns.isin(["geometry", "h3_id"])]
        if use_hexes_attr
        else []
    )

    hexes_y_columns_names = ["zabka"]

    X = hexes[hexes_attr_columns]
    y = controller.hexes_centroids_gdf[hexes_y_columns_names]

    return {"X": X, "y": y}


tabular_data_dict = {
    city_name: create_tabular_data(
        gdfs["hexes"], gdfs["controller"], use_ortophoto=True, use_hexes_attr=True
    )
    for city_name, gdfs in gdfs_dict.items()
}

# 10. Creating folds labels


In [19]:
def shift_elements_right(lst):
    shifted_lst = [lst[-1]] + lst[:-1]
    return shifted_lst


cities_names_list = list(graph_data_dict.keys())
cities_names_list.sort(key=lambda x: str(x))

# val + test
folds_tuples = list(zip(shift_elements_right(cities_names_list), cities_names_list))
display(folds_tuples)

[('Wrocław', 'Kraków'),
 ('Kraków', 'Poznań'),
 ('Poznań', 'Szczecin'),
 ('Szczecin', 'Warszawa'),
 ('Warszawa', 'Wrocław')]

# 11. Functions setup


In [20]:
def run_k_fold_graph_data(closure_config, sweep_id):
    # pass external config (i.e. what attributes are used in the data), closure to avoid passing it to the function directly
    def wrapped():
        run = wandb.init()
        epochs = EPOCHS

        config = wandb.config

        for k, v in closure_config.items():
            run.log({k: 1 if v else 0})

        run.log({"data_structure": "graph"})

        # create hparams
        if hasattr(config, "lin_layer_size") and hasattr(config, "num_lin_layers"):
            lin_layer_sizes = [config.lin_layer_size] * config.num_lin_layers
        else:
            lin_layer_sizes = config.lin_layer_sizes
        hparams = {
            "hidden_channels": config.hidden_channels,
            "lr": config.learning_rate,
            "num_conv_layers": config.num_conv_layers,
            "lin_layer_sizes": lin_layer_sizes,
            "weight_decay": config.weight_decay,
        }

        aucs = []
        accuracies = []
        f1s = []

        fold_group_id = generate_id()

        # log data as artifact if no data was logged in the sweep before
        # dataset is uploaded only on the first run in sweep, because it does not change across runs in sweep
        # in wandb, dataset will be visible on the first run in the sweep
        artifact_path = os.path.join(TRAIN_SAVE_DIR, f"graph_data_{sweep_id}.pkl")
        if not os.path.exists(artifact_path):
            dump(
                graph_data_dict,
                artifact_path,
                protocol=5,
            )
            artifact = wandb.Artifact(
                name="graph_data", type="dataset", metadata=closure_config
            )
            artifact.add_file(local_path=artifact_path)
            run.log_artifact(artifact)

        # run k-fold
        for index, (val_city_name, test_city_name) in enumerate(folds_tuples):
            # prepare data
            val_data = [graph_data_dict[val_city_name].to("cpu").clone()]
            train_data = [
                v.to("cpu").clone()
                for k, v in graph_data_dict.items()
                if k != val_city_name and k != test_city_name
            ]
            test_data = graph_data_dict[test_city_name].to("cpu").clone()

            # run training with checkpointing on lowest val_loss, return test metrics for the best model and its path
            # builtin preprocessing - scaling to N(0, 1)
            auc, accuracy, f1, model_path = train(
                train_data=train_data,
                val_data=val_data,
                test_data=test_data,
                epochs=epochs,
                hparams=hparams,
                train_save_dir=TRAIN_SAVE_DIR,
            )

            # logging - single fold
            run.log_model(
                path=model_path,
                name=f"model_{fold_group_id}_fold_{index}",
            )
            run.log({f"auc_fold_{index}": auc})
            run.log({f"accuracy_fold_{index}": accuracy})
            run.log({f"f1_fold_{index}": f1})

            aucs.append(auc)
            accuracies.append(accuracy)
            f1s.append(f1)

        # logging - summary statistics
        mean_auc = sum(aucs) / len(aucs)
        mean_accuracy = sum(accuracies) / len(accuracies)
        mean_f1 = sum(f1s) / len(f1s)
        run.log({"mean_auc": mean_auc})
        run.log({"mean_accuracy": mean_accuracy})
        run.log({"mean_f1": mean_f1})

    return wrapped


def run_k_fold_tabular_data(closure_config, sweep_id):
    # analogously to the graph data, but for tabular data
    def wrapped():
        run = wandb.init()

        config = wandb.config

        for k, v in closure_config.items():
            run.log({k: 1 if v else 0})

        run.log({"data_structure": "tabular"})

        hparams = {}
        hparams["C"] = config["C"]
        solver, penalty = config["solver_penalty"].split(";")
        hparams["solver"] = solver
        if penalty == "None":
            penalty = None
        hparams["penalty"] = penalty

        aucs = []
        accuracies = []
        f1s = []

        fold_group_id = generate_id()

        # log data as artifact
        artifact_path = os.path.join(TRAIN_SAVE_DIR, f"tabular_data_{sweep_id}.pkl")

        if not os.path.exists(artifact_path):
            dump(
                tabular_data_dict,
                artifact_path,
                protocol=5,
            )
            artifact = wandb.Artifact(
                name="tabular_data", type="dataset", metadata=closure_config
            )
            artifact.add_file(local_path=artifact_path)
            run.log_artifact(artifact)

        timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

        for index, test_city_name in enumerate(cities_names_list):
            scaler = StandardScaler()
            X = pd.concat(
                [
                    m["X"]
                    for key, m in tabular_data_dict.items()
                    if key != test_city_name
                ]
            ).to_numpy()
            y = (
                pd.concat(
                    [
                        m["y"]
                        for key, m in tabular_data_dict.items()
                        if key != test_city_name
                    ]
                )
                .to_numpy()
                .ravel()
            )

            X = scaler.fit_transform(X)

            logistic_regression = LogisticRegression(
                C=hparams["C"],
                solver=hparams["solver"],
                penalty=hparams["penalty"],
                dual=False,
                tol=1e-4,
                fit_intercept=True,
                intercept_scaling=1,
                class_weight="balanced",
                random_state=1124,
                max_iter=1000,
                multi_class="auto",
                warm_start=False,
                n_jobs=-1,
                l1_ratio=0.5,
            )
            logistic_regression.fit(X, y)

            test_X = tabular_data_dict[test_city_name]["X"].to_numpy()
            test_X = scaler.transform(test_X)
            test_y = tabular_data_dict[test_city_name]["y"].to_numpy().ravel()
            y_pred = logistic_regression.predict(test_X)
            y_proba = logistic_regression.predict_proba(test_X)[:, 1]

            auc = roc_auc_score(test_y, y_proba, average="micro")
            accuracy = (y_pred == test_y).mean()
            f1 = f1_score(
                test_y,
                y_pred,
                pos_label=1,
                average="binary",
            )

            model_dir = os.path.join(TRAIN_SAVE_DIR, timestamp)

            os.makedirs(model_dir, exist_ok=True)

            model_path = os.path.join(
                model_dir, f"model_{fold_group_id}_fold_{index}.pkl"
            )

            with open(model_path, "wb") as f:
                dump(logistic_regression, f, protocol=5)

            run.log_model(
                path=model_path,
                name=f"model_{fold_group_id}_fold_{index}",
            )
            run.log({f"auc_fold_{index}": auc})
            run.log({f"accuracy_fold_{index}": accuracy})
            run.log({f"f1_fold_{index}": f1})

            aucs.append(auc)
            accuracies.append(accuracy)
            f1s.append(f1)

        mean_auc = sum(aucs) / len(aucs)
        mean_accuracy = sum(accuracies) / len(accuracies)
        mean_f1 = sum(f1s) / len(f1s)
        run.log({"mean_auc": mean_auc})
        run.log({"mean_accuracy": mean_accuracy})
        run.log({"mean_f1": mean_f1})

    return wrapped


def run_sweep_graph_data(config):
    try:
        wandb.login(key=WANDB_API_KEY)
        sweep_id = wandb.sweep(
            WANDB_SWEEP_PARAMS_GRAPH_DATA, project="zabka-downstream-task"
        )

        wandb.agent(
            sweep_id,
            function=run_k_fold_graph_data(config, sweep_id),
            count=SWEEP_RUNS_COUNT,
        )
    except Exception as e:
        print(e)
        wandb.finish()
        wandb.sweep
        raise e


def run_sweep_tabular_data(config):
    try:
        wandb.login(key=WANDB_API_KEY)

        sweep_id = wandb.sweep(
            WANDB_SWEEP_PARAMS_TABULAR_DATA, project="zabka-downstream-task"
        )

        wandb.agent(
            sweep_id,
            function=run_k_fold_tabular_data(config, sweep_id),
            count=SWEEP_RUNS_COUNT,
        )
    except Exception as e:
        print(e)
        wandb.finish()
        raise e

# 12. Run functions

For each config:

1. Determine if config requires tabular or graph data
2. Create data excluding attributes not included in the config
3. Run the sweep


In [21]:
def derive_data_structure(attr_config):
    if attr_config["USE_OSMNX_ATTRS"]:
        return "graph"
    return "tabular"


configs_size = len(ATTRIBUTES_CONFIGURATIONS)

for index, attr_config in enumerate(ATTRIBUTES_CONFIGURATIONS):
    print("Sweep for config {}/{} in progress...".format(index + 1, configs_size))

    assert "USE_ORTOPHOTO" in attr_config, "Provide USE_ORTOPHOTO key"
    assert "USE_HEXES_ATTRS" in attr_config, "Provide USE_HEXES_ATTRS key"
    assert "USE_OSMNX_ATTRS" in attr_config, "Provide USE_OSMNX_ATTRS key"

    data_structure = derive_data_structure(attr_config)

    if data_structure == "graph":
        graph_data_dict = {
            city_name: create_graph_data(
                **gdfs,
                use_ortophoto=attr_config["USE_ORTOPHOTO"],
                use_hexes_attr=attr_config["USE_HEXES_ATTRS"],
            )
            for city_name, gdfs in gdfs_dict.items()
        }
        run_sweep_graph_data(attr_config)
    elif data_structure == "tabular":
        tabular_data_dict = {
            city_name: create_tabular_data(
                gdfs["hexes"],
                gdfs["controller"],
                use_ortophoto=attr_config["USE_ORTOPHOTO"],
                use_hexes_attr=attr_config["USE_HEXES_ATTRS"],
            )
            for city_name, gdfs in gdfs_dict.items()
        }
        run_sweep_tabular_data(attr_config)
    else:
        raise ValueError("Unknown data structure")

Sweep for config 1/2 in progress...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: juliajaniak (gradient_pwr). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/jjaniak/.netrc


Create sweep with ID: d0rbt10k
Sweep URL: https://wandb.ai/gradient_pwr/zabka-downstream-task/sweeps/d0rbt10k


wandb: Agent Starting Run: 81ergqrv with config:
wandb: 	hidden_channels: 20
wandb: 	learning_rate: 0.0050065230358305665
wandb: 	lin_layer_size: 32
wandb: 	num_conv_layers: 2
wandb: 	num_lin_layers: 0
wandb: 	weight_decay: 2.9110922701160134e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../../gradient_logs/2024_06_04_11_52_15/lightning_logs
/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/pytorch_lightning/utilities/model_summary/model_summary.py:452: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.

  | Name  | Type      | Params
------------------------------------
0 | model | HeteroGNN | 90.9 K
------------------------------------
90.9 K    Trainable params
0         Non-trainable params
90.9 K    Total params
0.364     Total estimated model params size (MB)


...

/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../../gradient_logs/2024_06_04_11_56_04/lightning_logs

  | Name  | Type      | Params
------------------------------------
0 | model | HeteroGNN | 8.2 K 
------------------------------------
8.2 K     Trainable params
0         Non-trainable params
8.2 K     Total params
0.033     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


USE_HEXES_ATTRS,▁
USE_ORTOPHOTO,▁
USE_OSMNX_ATTRS,▁
USE_HEXES_ATTRS,0
USE_ORTOPHOTO,0
USE_OSMNX_ATTRS,1
data_structure,graph


Run o4uqmb73 errored:
Traceback (most recent call last):
  File "/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/var/folders/4k/8js3jm9n457cm4_v631zyrkw0000gn/T/ipykernel_37726/1604121319.py", line 62, in wrapped
    auc, accuracy, f1, model_path = train(
  File "/Users/jjaniak/Documents/studia/projekt/gradient/notebooks/downstream_tasks/../../src/training/train.py", line 64, in train
    trainer.fit(model, train_loader, val_loader)
  File "/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 545, in fit
    call._call_and_handle_interrupt(
  File "/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py", line 44, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/pytorch_li